In [1]:
import os
from preprocessing import *
import torch
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from torchvision import transforms
from torchvision.transforms import Compose, Resize, ToTensor, Grayscale

c:\Users\Aina\miniconda3\envs\tfg_aina\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_PATH = "c:\\Users\\Aina\\OneDrive\\Escritorio\\White-Matter-Hyperintensity-Segmentation"+ "\\ICPR"
TRAINING_PATH= BASE_PATH + "\\train\\"

TEST_PATH= BASE_PATH + "/test/"

In [3]:
TRAINING_DICT={
    "pathsFLAIR":[],
    "pathsT1":[],
    "pathsT2":[],
    "mask" :[]
}
createDictionary(TRAINING_PATH, TRAINING_DICT)
print(TRAINING_DICT.get("pathsFLAIR").__getitem__(0))
print(TRAINING_DICT.get("pathsT1").__getitem__(0))
print(TRAINING_DICT.get("pathsT2").__getitem__(0))
print(TRAINING_DICT.get("mask").__getitem__(0))
print(len(TRAINING_DICT.get("pathsFLAIR")))

c:\Users\Aina\OneDrive\Escritorio\White-Matter-Hyperintensity-Segmentation\ICPR\train\P1\T1\P1_T1_FLAIR.nii.gz
c:\Users\Aina\OneDrive\Escritorio\White-Matter-Hyperintensity-Segmentation\ICPR\train\P1\T1\P1_T1_T1.nii.gz
c:\Users\Aina\OneDrive\Escritorio\White-Matter-Hyperintensity-Segmentation\ICPR\train\P1\T1\P1_T1_T2.nii.gz
c:\Users\Aina\OneDrive\Escritorio\White-Matter-Hyperintensity-Segmentation\ICPR\train\P1\T1\P1_T1_MASK.nii.gz
93


In [4]:
# TEST_DICT={
#     "pathsFLAIR":[],
#     "pathsT1":[],
#     "pathsT2":[],
#     "mask" :[],
# }
# createDictionary(TEST_PATH, TEST_DICT)
# print(TEST_DICT.get("pathsFLAIR").__getitem__(0))
# print(TEST_DICT.get("pathsT1").__getitem__(0))
# print(TEST_DICT.get("pathsT2").__getitem__(0))
# print(TEST_DICT.get("mask").__getitem__(0))

# DATASET

In [5]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128), antialias=True),
    transforms.Grayscale()
])

transform_label=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128), antialias=True, interpolation= transforms.InterpolationMode.NEAREST),
])

In [6]:
train,val = getIDsICPR(TRAINING_DICT)
train_data = ICPR(train.get('pathsFLAIR'), train.get('pathsT1'),train.get('pathsT2'), train.get('mask'),transform, transform_label, True)
val_data = ICPR(val.get('pathsFLAIR'), val.get('pathsT1'),val.get('pathsT2'), val.get('mask'),transform, transform_label, True)

In [7]:
print(len(train.get('pathsFLAIR')))
print(len(val.get('pathsFLAIR')))

74
19


In [8]:
print(len(train_data.images))
print(len(val_data.images))

74
19


In [9]:
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
val_loader = DataLoader(val_data, batch_size=2, shuffle=False)

# V-NET

In [11]:
import vnet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = vnet.VNet(128, 2)
model = model.to(device)
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

RuntimeError: Given transposed=1, weight of size [192, 96, 2, 2, 2], expected input[1, 128, 11, 13, 11] to have 192 channels, but got 128 channels instead

In [ ]:
# Training loop
num_epochs = 1
epoch_loss_values = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_num, (input_img, target) in enumerate(train_loader, 1):   
        
        step += 1
        input_img= input_img.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_img)
        loss = loss_function(outputs, target)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        print(f"{step}/{len(train_loader)} - Loss: {loss.item():.4f}")
    
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"Epoch {epoch+1} average loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "vnet_model.pth")


Epoch 1/1


TypeError: Input image tensor permitted channel values are [1, 3], but found 182